In [ ]:
import pandas as pd

dataset = 'ml-latest-small'

ratings = pd.read_csv('../datasets/' + dataset + '/ratings.csv')
total_movies = len(list(set(ratings.movieId.to_list())))
max_ratings = max(list(set(ratings.rating.to_list())))

In [ ]:
from random import randrange

# add random ratings to first user in cluster 0
target_user = 601
N = 350

# user_ratings_df = ratings[ratings['userId'] == target_user]

for n in range(N):
    # new record to add
    new_record = {
        'userId': target_user,
        'movieId': randrange(total_movies),
        'rating': randrange(max_ratings),
        'timestamp': 1212603770
    }
    # append row to the dataframe
    ratings = ratings.append(new_record, ignore_index=True)

In [ ]:
ratings

In [ ]:
ratings.to_csv('../datasets/ml-latest-small/ratings_random.csv', index=False)